# Introduction to GeoPandas

[Jian Tao](https://coehpc.engr.tamu.edu/people/jian-tao/), and [Enrique Z. Losoya](https://orcid.org/0000-0001-7763-3349), Texas A&M University. 

Updated: January 3, 2023.

Converted from 
[GeoHackWeek 2019](https://geohackweek.github.io) -- [vector tutorial](https://geohackweek.github.io/vector/)   
[Emilio Mayorga, University of Washington](https://github.com/emiliom/). 2019-9-9

## 1. Introduction

[GeoPandas](http://geopandas.org) adds a spatial geometry data type to `Pandas` and enables spatial operations on these types, using [shapely](http://toblerity.org/shapely/). GeoPandas leverages Pandas together with several core open source geospatial packages and practices to provide a uniquely simple and convenient framework for handling geospatial feature data, operating on both geometries and attributes jointly, and as with Pandas, largely eliminating the need to iterate over features (rows). Also as with Pandas, it adds a very convenient and fine-tuned plotting method, and read/write methods that handle multiple geospatial file and "serialization" formats.

In [ ]:
!pip install geopandas

## 2. Import packages, and set data file path
We'll use these throughout the rest of the tutorial.

In [ ]:
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

In [ ]:
mpl.__version__, pd.__version__, gpd.__version__

In [ ]:
data_pth = Path("../data")

## 3. GeoSeries: The geometry building block

Like a Pandas `Series`, a `GeoSeries` is the building block for the more broadly useful and powerful `GeoDataFrame` that we'll focus on in this tutorial. Here we'll first take a bit of time to examine a `GeoSeries`.

A `GeoSeries` is made up of an index and a GeoPandas `geometry` data type. This data type is a [shapely.geometry object](http://toblerity.org/shapely/manual.html#geometric-objects), and therefore inherits their attributes and methods such as `area`, `bounds`, `distance`, etc.

GeoPandas has six classes of **geometric objects**, corresponding to the three basic single-entity geometric types and their associated homogeneous collections of multiple entities:
- **Single entity (core, basic types):**
  - Point
  - Line (*formally known as a LineString*)
  - Polygon
- **Homogeneous entity collections:**
  - Multi-Point
  - Multi-Line (*MultiLineString*)
  - Multi-Polygon

A `GeoSeries` is then a list of geometry objects and their associated index values.

<div class="alert alert-warning" style="font-size:100%">
<b style="font-size:120%">Entries (rows) in a GeoSeries can store different geometry types!</b></br></br>

GeoPandas does not constrain the geometry column to be of the same geometry type. This can lead to unexpected problems if you're not careful! Specially if you're used to thinking of a GIS file format like shape files, which store a single geometry type. Also beware that certain export operations (say, to shape files ...) will fail if the list of geometry objects is heterogeneous.
</div>

But enough theory! Let's get to the code. We'll start by illustrating how GeoSeries are constructured.

### Create a `GeoSeries` from a list of `shapely Point` objects using the `Point` constructor

In [ ]:
gs = GeoSeries([Point(-120, 45), Point(-121.2, 46), Point(-122.9, 47.5)])
gs

In [ ]:
type(gs), len(gs)

Occasionally -- but rarely -- you may encounter the need to parse and convert a geometry found as a `WKT` string. A `shapely Point` object can be constructed from it using`shapely.wkt.loads`:
```python
from shapely.wkt import loads
loads('POINT(-121.2 46)')
```

A GeoSeries (and a GeoDataFrame) can store a CRS implicitly associated with the geometry column. This is useful as essential spatial metadata and for transformation (reprojection) to another CRS. Let's assign the CRS, then plot it.

In [ ]:
gs.crs = {'init': 'epsg:4326'}

In [ ]:
gs.plot()

The `plot` method accepts standard `matplotlib.pyplot` style options, and can be tweaked like any other `matplotlib` figure.

In [ ]:
gs.plot(marker='*', color='red', markersize=100, figsize=(4, 4))
plt.xlim([-123, -119.8])
plt.ylim([44.8, 47.7]);

**Let's get a bit fancier, as a stepping stone to GeoDataFrames.** First, we'll define a simple dictionary of lists, that we'll use again later.

In [ ]:
data = {'name': ['a', 'b', 'c'],
        'lat': [45, 46, 47.5],
        'lon': [-120, -121.2, -122.9]}

Note this convenient, compact approach to create a list of `Point` shapely objects out of X & Y coordinate lists using the `gpd.points_from_xy` convenience method (introduced in version 0.5.0). The same result could be obtained with the more verbose form `geometry = [Point(x,y) for x,y in zip(data['lon'], data['lat'])]`.

In [ ]:
geometry = gpd.points_from_xy(data['lon'], data['lat'])

We'll wrap up by creating a GeoSeries where we explicitly define the index values.

In [ ]:
gs = GeoSeries(geometry, index=data['name'])
gs

## 4. GeoDataFrames: The real power tool

<div class="alert alert-info" style="font-size:100%">

It's worth noting that a GeoDataFrame can be described as a **Feature Collection**, where each row is a *Feature*, a *geometry* column is defined (thought the name of the column doesn't have to be "geometry"), and the attribute *properties* includes the other columns (the Pandas DataFrame part, if you will).

</div>

<div class="alert alert-warning" style="font-size:100%">

More than one column can store geometry objects! We won't explore this capability in this tutorial.

</div>

### Start with a simple, manually constructed illustration

We'll build on the GeoSeries examples. Let's reuse the `data` dictionary we defined earlier, this time to create a DataFrame.

In [ ]:
df = pd.DataFrame(data)
df

Now we use the DataFrame and `points_from_xy` (which creates a list of shapely Point objects) to create a GeoDataFrame. Note the use of two DataFrame attribute columns, which are effectively just two simple Pandas Series.

In [ ]:
gdf = GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))

In [ ]:
gdf

There's nothing new to visualize, but this time we're using the `plot` method from a GeoDataFrame, *not* from a GeoSeries. They're not exactly the same thing under the hood.

In [ ]:
gdf.plot(marker='*', color='green', markersize=50, figsize=(3, 3));

### FINALLY, we get to work with real data! Load and examine the simple "oceans" shape file

`gpd.read_file` is the workhorse for reading GIS files. It leverages the [fiona](http://toblerity.org/fiona/README.html) package.

In [ ]:
!wget https://github.com/geohackweek/tutorial_contents/raw/master/vector/data/oceans.shp
!wget https://github.com/geohackweek/tutorial_contents/raw/master/vector/data/oceans.shx

In [ ]:
oceans = gpd.read_file("./oceans.shp")

In [ ]:
oceans.head()

The `crs` was read from the shape file's `prj` file:

In [ ]:
oceans.crs

Now we finally plot a real map (or blobs, depending on your aesthetics), from a dataset that's global-scale and stored in "geographic" (latitude & longitude) coordinates. It's *not* the actual ocean shapes defined by coastal boundaries, but bear with me. A colormap has been applied to distinguish the different "oceans".

In [ ]:
oceans.plot(cmap='Set2', figsize=(10, 10));

`oceans.shp` stores both `Polygon` and `Multi-Polygon` geometry types (a `Polygon` may also be viewed as a `Multi-Polygon` with 1 member). We can get at the geometry types and other geometry properties easily.

In [ ]:
oceans.geom_type

In [ ]:
# Beware that these area calculations are in the native degrees units, which is fairly useless
oceans.geometry.area

In [ ]:
oceans.geometry.bounds

The `envelope` method returns the bounding box for each polygon. This could be used to create a new spatial column or GeoSeries; directly for plotting; etc.

In [ ]:
oceans.envelope.plot(cmap='Set2', figsize=(8, 8), alpha=0.7, edgecolor='black');

Does it seem weird that some envelope bounding boxes, such as the North Pacific Ocean, span all longitudes? That's because they're Multi-Polygons with edges at the ends of the -180 and +180 degree coordinate range.

### Load "Natural Earth" countries dataset, bundled with GeoPandas
*"[Natural Earth](http://www.naturalearthdata.com) is a public domain map dataset available at 1:10m, 1:50m, and 1:110 million scales. Featuring tightly integrated vector and raster data, with Natural Earth you can make a variety of visually pleasing, well-crafted maps with cartography or GIS software."* A subset comes bundled with GeoPandas and is accessible from the `gpd.datasets` module. We'll use it as a helpful global base layer map.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head(2)

Its CRS is also EPSG:4326:

In [ ]:
world.crs

In [ ]:
world.plot(figsize=(8, 8));

### Map plot overlays: Plotting multiple spatial layers

Here's a compact, quick way of using the GeoDataFrame plot method to overlay two GeoDataFrames while customizing the styles for each layer. The "inner" plot (`oceans`) is displayed in the background.

In [ ]:
world.plot(ax=oceans.plot(cmap='Set2', figsize=(10, 10)), 
           facecolor='gray');

We can also compose the plot using conventional `matplotlib` steps and options that give us more control.

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 6))
ax.set_title('Countries and Ocean Basins')
# Other nice categorical color maps (cmap) include 'Set2' and 'Set3'
oceans.plot(ax=ax, cmap='Paired')
world.plot(ax=ax, facecolor='lightgray', edgecolor='gray')
ax.set_ylim([-90, 90])
ax.set_axis_off()
plt.axis('equal');

<div class="alert alert-success" style="font-size:100%">
<b style="font-size:120%">Time to Explore</b></br>

Let's stop for a bit to explore on your own, hack with your neighbors, ask questions.

</div>